In [1]:
import pandas as pd
import numpy as np
import joblib
from scipy.signal import butter, filtfilt
from scipy import stats
from collections import Counter

In [2]:
import sys
import os
sys.path.insert(0, os.path.abspath('.'))

from lib.dobot import Dobot
bot = Dobot('COM3')

ModuleNotFoundError: No module named 'serial'

# Neue Daten einlesen
Hier werden die neuen Daten eingelesen.

In [ ]:
new_data = pd.read_csv("RawDataWisch.csv")

# Filtern der Daten
Hier werden die Daten gefiltert

In [ ]:
filter_order = 4
low_cutoff = 0.5
high_cutoff = 5
fs = 200
b, a = butter(filter_order, [low_cutoff, high_cutoff], btype='band', fs=fs)

gx_col = 'Linear Acceleration x (m/s^2)'
gy_col = 'Linear Acceleration y (m/s^2)'
gz_col = 'Linear Acceleration z (m/s^2)'

# Filter anwenden
new_data['filtered_X_band'] = filtfilt(b, a, new_data[gx_col])
new_data['filtered_Y_band'] = filtfilt(b, a, new_data[gy_col])
new_data['filtered_Z_band'] = filtfilt(b, a, new_data[gz_col])

# Z-Score Ausreißer entfernen
z_score_threshold = 4
new_data = new_data[(abs(stats.zscore(new_data[gx_col])) <= z_score_threshold) &
                    (abs(stats.zscore(new_data[gy_col])) <= z_score_threshold) &
                    (abs(stats.zscore(new_data[gz_col])) <= z_score_threshold)]

# Windowing

In [ ]:
window_size = 100
overlap = 50
def create_windows(df):
    windows = []
    for i in range(0, len(df), window_size - overlap):
        window = df.iloc[i:i+window_size]
        if len(window) == window_size:
            windows.append(window)
    return windows

windows = create_windows(new_data)

# Fenster erstellen

In [ ]:
def extract_features(windows):
    feature_list = []
    for window in windows:
        features = [
            window['filtered_X_band'].mean(),
            window['filtered_Y_band'].mean(),
            window['filtered_Z_band'].mean(),
            window['filtered_X_band'].std(),
            window['filtered_Y_band'].std(),
            window['filtered_Z_band'].std(),
            window['filtered_X_band'].max(),
            window['filtered_Y_band'].max(),
            window['filtered_Z_band'].max(),
            window['filtered_X_band'].min(),
            window['filtered_Y_band'].min(),
            window['filtered_Z_band'].min()
        ]
        feature_list.append(features)
    return pd.DataFrame(feature_list, columns=[
        'BeschleunigungX_Mittelwert', 'BeschleunigungY_Mittelwert', 'BeschleunigungZ_Mittelwert',
        'BeschleunigungX_Std', 'BeschleunigungY_Std', 'BeschleunigungZ_Std',
        'BeschleunigungX_Max', 'BeschleunigungY_Max', 'BeschleunigungZ_Max',
        'BeschleunigungX_Min', 'BeschleunigungY_Min', 'BeschleunigungZ_Min'
    ])

X_features_df = extract_features(windows)

# Entscheidungsbaum laden
Hier wird der Entscheidungsbaum geladen

In [ ]:
dt = joblib.load("decision_tree_model.pkl")

# Vorhersagen
Hier wird Vorhergesagt, welche Bewegung durchgeführt wurde. 

In [ ]:
# Vorhersage für alle Fenster
y_pred = dt.predict(X_features_df)

# Gesamtbewegung bestimmen
overall_pred = Counter(y_pred).most_common(1)[0][0]
print("Gesamtbewegung erkannt:", overall_pred)

# Funktion für die Bewegung (Wischen)
Diese Funktion wird ausgeführt, wenn eine Wisch Bewegung erkannt wurde.

In [ ]:
def wischen(bot):
    print("Aktion: Wischen")
    x_min = 220
    x_haken = 260
    x_max = 300

    y_min = -80
    y_haken = -40
    y_max = -20

    z_min = -72.5
    z_max = -8

    print('Hochfahren')
    bot.move_to(x_haken, 0, z_max, 0)

    print('Homing-Modus')
    bot.home()

    print('Kastenstartposition')
    bot.move_to(x_max, y_min, z_max, 0)

    print('Kastenposition 1')
    bot.move_to(x_max, y_min, z_min, 0)

    print('Kastenposition 2')
    bot.slide_to(x_min, y_min, z_min, 0)

    print('Kastenposition 3')
    bot.slide_to(x_min, y_max, z_min, 0)

    print('Kastenposition 4')
    bot.slide_to(x_max, y_max, z_min, 0)

    print('Kastenposition 1')
    bot.slide_to(x_max, y_min, z_min, 0)

    print('Hochfahren')
    bot.move_to(x_max, y_min, z_max, 0)

    print('Hakenstartposition')
    bot.move_to(x_haken, y_max, z_max, 0)

    print('Position 1')
    bot.move_to(x_haken, y_max, z_min, 0)

    print('Position 2')
    bot.slide_to(x_min, y_haken, z_min, 0)

    print('Position 3')
    bot.slide_to(x_max, y_min, z_min, 0)

    print('Zurück nach Position 1')
    bot.move_to(x_max, y_min, z_max, 0)

# Funktion für die Bewegung (Hammer)
Diese Funktion wird ausgeführt, wenn eine Hammer Bewegung erkannt wurde.

In [ ]:
def hammer(bot):

    print("Aktion: Hammer")
    x_min = 220
    x_max = 300

    y_min = 20
    y_max = 80

    z_min = -72.5
    z_max = -8

    print('Hochfahren')
    bot.move_to(x_max, 0, z_max, 0)

    print('Homing-Modus')
    bot.home()

    print('Startposition')
    bot.move_to(x_max, y_min, z_max, 0)

    print('Kasten-Position 1')
    bot.move_to(x_max, y_min, z_min, 0)

    print('Kasten-Position 2')
    bot.slide_to(x_min, y_min, z_min, 0)

    print('Kasten-Position 3')
    bot.slide_to(x_min, y_max, z_min, 0)

    print('Kasten-Position 4')
    bot.slide_to(x_max, y_max, z_min, 0)

    print('Kasten-Position 1')
    bot.slide_to(x_max, y_min, z_min, 0)

    print('Kreuz-Position 1')
    bot.slide_to(x_min, y_max, z_min, 0)

    print('Hochfahren 2')
    bot.move_to(x_min, y_max, z_max, 0)

    print('Startposition 2')
    bot.move_to(x_max, y_max, z_max, 0)

    print('Kreuz-Position 2')
    bot.move_to(x_max, y_max, z_min, 0)

    print('Kreuz-Position 3')
    bot.slide_to(x_min, y_min, z_min, 0)

    print('Hochfahren 3')
    bot.move_to(x_min, y_min, z_max, 0)

## Dobot Aktionen anhand der Vorhersage ausführen

Hier verknüpfen wir die prediction mit dem Bot

In [ ]:
if overall_pred == "Hammer":
    hammer(bot)
else:
    wischen(bot)

## Wenn der Code oben nicht funktioniert mit dem Roboter

In [ ]:
import sys
import os
sys.path.insert(0, os.path.abspath('.'))

from lib.dobot import Dobot

bot = Dobot('COM3')

def wischen(bot):
    print("Aktion: Wischen")
    x_min = 220
    x_haken = 260
    x_max = 300

    y_min = -80
    y_haken = -40
    y_max = -20

    z_min = -72.5
    z_max = -8

    print('Hochfahren')
    bot.move_to(x_haken, 0, z_max, 0)

    print('Homing-Modus')
    bot.home()

    print('Kastenstartposition')
    bot.move_to(x_max, y_min, z_max, 0)

    print('Kastenposition 1')
    bot.move_to(x_max, y_min, z_min, 0)

    print('Kastenposition 2')
    bot.slide_to(x_min, y_min, z_min, 0)

    print('Kastenposition 3')
    bot.slide_to(x_min, y_max, z_min, 0)

    print('Kastenposition 4')
    bot.slide_to(x_max, y_max, z_min, 0)

    print('Kastenposition 1')
    bot.slide_to(x_max, y_min, z_min, 0)

    print('Hochfahren')
    bot.move_to(x_max, y_min, z_max, 0)

    print('Hakenstartposition')
    bot.move_to(x_haken, y_max, z_max, 0)

    print('Position 1')
    bot.move_to(x_haken, y_max, z_min, 0)

    print('Position 2')
    bot.slide_to(x_min, y_haken, z_min, 0)

    print('Position 3')
    bot.slide_to(x_max, y_min, z_min, 0)

    print('Zurück nach Position 1')
    bot.move_to(x_max, y_min, z_max, 0)

def hammer(bot):

    print("Aktion: Hammer")
    x_min = 220
    x_max = 300

    y_min = 20
    y_max = 80

    z_min = -72.5
    z_max = -8

    print('Hochfahren')
    bot.move_to(x_max, 0, z_max, 0)

    print('Homing-Modus')
    bot.home()

    print('Startposition')
    bot.move_to(x_max, y_min, z_max, 0)

    print('Kasten-Position 1')
    bot.move_to(x_max, y_min, z_min, 0)

    print('Kasten-Position 2')
    bot.slide_to(x_min, y_min, z_min, 0)

    print('Kasten-Position 3')
    bot.slide_to(x_min, y_max, z_min, 0)

    print('Kasten-Position 4')
    bot.slide_to(x_max, y_max, z_min, 0)

    print('Kasten-Position 1')
    bot.slide_to(x_max, y_min, z_min, 0)

    print('Kreuz-Position 1')
    bot.slide_to(x_min, y_max, z_min, 0)

    print('Hochfahren 2')
    bot.move_to(x_min, y_max, z_max, 0)

    print('Startposition 2')
    bot.move_to(x_max, y_max, z_max, 0)

    print('Kreuz-Position 2')
    bot.move_to(x_max, y_max, z_min, 0)

    print('Kreuz-Position 3')
    bot.slide_to(x_min, y_min, z_min, 0)

    print('Hochfahren 3')
    bot.move_to(x_min, y_min, z_max, 0)

if overall_pred == "Wischen":
    wischen(bot)
else:
    hammer(bot)